# Docker Image
We create a docker image to be used to run `TrainTestClassify.py` on the Batch AI cluster.

In [1]:
import os
from os import path
import json
import shutil
import dotenv
%load_ext dotenv

Create a file to contain the name of the docker login and image name to be used.

In [2]:
dotenv_path = os.path.join('.', '.env')  # The location of the dotenv file
if os.path.isfile(dotenv_path):          # Remove any pre-existing dotenv file to ensure a blank slate
    os.remove(dotenv_path)
with open(dotenv_path, 'w'):             # Create an empty dotenv file
    None
# Your docker login and image repository name
dotenv.set_key(dotenv_path, 'docker_login', 'YOUR_DOCKER_LOGIN')
dotenv.set_key(dotenv_path, 'image_repo', '/mlbaiht')

(True, 'image_repo', '/mlbaiht')

In [8]:
dotenv_path = os.path.join('.', '.env')  # The location of the dotenv file
if os.path.isfile(dotenv_path):          # Remove any pre-existing dotenv file to ensure a blank slate
    os.remove(dotenv_path)
with open(dotenv_path, 'w'):             # Create an empty dotenv file
    None
# Your docker login and image repository name
dotenv.set_key(dotenv_path, 'docker_login', 'mabouatmicrosoft')
dotenv.set_key(dotenv_path, 'image_repo', '/mlbaiht')

(True, 'image_repo', '/mlbaiht')

Import the contents of the file into the environment

In [9]:
%dotenv -o

Create the Docker directory.

In [4]:
!mkdir -p Docker

Add to the directory the conda environment file that specifies the virtual environment for the training script. This includes the Python modules needed to run the training script.

In [5]:
%%writefile Docker/environment.yml

name: MLBatchAIHyperparameterTuning
channels:
  - conda-forge
dependencies:
  - python=3.5
  - pandas==0.23.4
  - scikit-learn==0.19.1
  - pip:
    - lightgbm==2.1.2


Overwriting Docker/environment.yml


Add to the directory the dockerfile specifying the build.

In [6]:
%%writefile Docker/Dockerfile

# Start from a Ubuntu image
FROM ubuntu:16.04

# The miniconda version to install, and the name of the virtual environment.
ENV MINICONDA3_VERSION=4.5.4 ENVIRONMENT=MLBatchAIHyperparameterTuning

# Update the Ubuntu image, and install downloading tools.
RUN apt-get update --fix-missing \
    && apt-get install -y --no-install-recommends \
        bzip2 \
        ca-certificates \
        curl \
        git \
        wget \
    && apt-get clean \
    && rm -rf /var/lib/apt/lists/*

# Copy into the image the definition of the virtual environment.
COPY environment.yml .

# Install miniconda, use it to create the virtual environment, and cause it to activate when the shell opens.
RUN wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-${MINICONDA3_VERSION}-Linux-x86_64.sh -O ~/miniconda.sh \
    && /bin/bash ~/miniconda.sh -b -p /opt/conda \
    && rm ~/miniconda.sh \
    && /opt/conda/bin/conda env create -f environment.yml -n ${ENVIRONMENT} \
    && /opt/conda/bin/conda clean -tipsy \
    && ln -s /opt/conda/etc/profile.d/conda.sh /etc/profile.d/conda.sh \
    && echo ". /opt/conda/etc/profile.d/conda.sh" >> ~/.bashrc \
    && echo "conda activate" ${ENVIRONMENT} >> ~/.bashrc


Overwriting Docker/Dockerfile


Create the docker image. This step can take two to three minutes.

In [11]:
%%time
image_name = os.getenv('docker_login') + os.getenv('image_repo')
print('Creating Docker image {}'.format(image_name))
!docker build -t $image_name Docker --no-cache

Creating Docker image mabouatmicrosoft/mlbaiht
Sending build context to Docker daemon  4.096kB
Step 1/5 : FROM ubuntu:16.04
 ---> b9e15a5d1e1a
Step 2/5 : ENV MINICONDA3_VERSION=4.5.4 ENVIRONMENT=MLBatchAIHyperparameterTuning
 ---> Running in f93f19fc8fb1
Removing intermediate container f93f19fc8fb1
 ---> b8a3635e2936
Step 3/5 : RUN apt-get update --fix-missing     && apt-get install -y --no-install-recommends         bzip2         ca-certificates         curl         git         wget     && apt-get clean     && rm -rf /var/lib/apt/lists/*
 ---> Running in c1d5868d06a2
Get:1 http://security.ubuntu.com/ubuntu xenial-security InRelease [107 kB]
Get:2 http://security.ubuntu.com/ubuntu xenial-security/universe Sources [91.2 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Get:4 http://security.ubuntu.com/ubuntu xenial-security/main amd64 Packages [707 kB]
Get:5 http://security.ubuntu.com/ubuntu xenial-security/restricted amd64 Packages [12.7 kB]
Get:6 http://security.ubu

Get:42 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 git amd64 1:2.7.4-0ubuntu1.4 [3158 kB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 15.9 MB in 1s (10.9 MB/s)
Selecting previously unselected package libgdbm3:amd64.
(Reading database ... 4768 files and directories currently installed.)
Preparing to unpack .../libgdbm3_1.8.3-13.1_amd64.deb ...
Unpacking libgdbm3:amd64 (1.8.3-13.1) ...
Selecting previously unselected package perl-modules-5.22.
Preparing to unpack .../perl-modules-5.22_5.22.1-9ubuntu0.5_all.deb ...
Unpacking perl-modules-5.22 (5.22.1-9ubuntu0.5) ...
Selecting previously unselected package libperl5.22:amd64.
Preparing to unpack .../libperl5.22_5.22.1-9ubuntu0.5_amd64.deb ...
Unpacking libperl5.22:amd64 (5.22.1-9ubuntu0.5) ...
Selecting previously unselected package perl.
Preparing to unpack .../perl_5.22.1-9ubuntu0.5_amd64.deb ...
Unpacking perl (5.22.1-9ubuntu0.5) ...
Selecting previously unselected package bzip2.
Prepa

Processing triggers for libc-bin (2.23-0ubuntu10) ...
Setting up libgdbm3:amd64 (1.8.3-13.1) ...
Setting up perl-modules-5.22 (5.22.1-9ubuntu0.5) ...
Setting up libperl5.22:amd64 (5.22.1-9ubuntu0.5) ...
Setting up perl (5.22.1-9ubuntu0.5) ...
update-alternatives: using /usr/bin/prename to provide /usr/bin/rename (rename) in auto mode
Setting up bzip2 (1.0.6-8) ...
Setting up libexpat1:amd64 (2.1.0-7ubuntu0.16.04.3) ...
Setting up libffi6:amd64 (3.2.1-4) ...
Setting up libgmp10:amd64 (2:6.1.0+dfsg-2) ...
Setting up libnettle6:amd64 (3.2-1ubuntu0.16.04.1) ...
Setting up libhogweed4:amd64 (3.2-1ubuntu0.16.04.1) ...
Setting up libidn11:amd64 (1.32-3ubuntu1.2) ...
Setting up libp11-kit0:amd64 (0.23.2-5~ubuntu16.04.1) ...
Setting up libtasn1-6:amd64 (4.7-3ubuntu0.16.04.3) ...
Setting up libgnutls30:amd64 (3.4.10-4ubuntu1.4) ...
Setting up libsqlite3-0:amd64 (3.11.0-1ubuntu1) ...
Setting up libssl1.0.0:amd64 (1.0.2g-1ubuntu4.13) ...
debconf: unable to initialize frontend: Dialog
debconf: (TER


#
# To activate this environment, use:
# > source activate MLBatchAIHyperparameterTuning
#
# To deactivate an active environment, use:
# > source deactivate
#

Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
six-1.11.0-py36h372c433_1.tar.bz2             21 KB
ncurses-6.1-hfc679d8_1.tar.bz2               1.2 MB
openssl-1.0.2o-h20670df_0.tar.bz2            3.4 MB
requests-2.18.4-py36he2e5f8d_1.tar.bz2        92 KB
numpy-1.15.1-py35_blas_openblashd3ea46f_1.tar.bz2     9.2 MB
pip-10.0.1-py36_0.tar.bz2                    1.8 MB
certifi-2018.4.16-py36_0.tar.bz2             142 KB
pysocks-1.6.8-py36_0.tar.bz2                  22 KB
blas-1.1-openblas.tar.bz2                      1 KB
tk-8.6.7-hc745277_3.tar.bz2                  3.2 MB
xz-5.2.4-h14c3975_4.tar.bz2                  366 KB
bzip2-1.0.6-h470a237_2.tar.bz2               310 KB
openssl-1.0.2p-h470a237_0.tar.bz2            3.5 MB
libgcc-ng-7.2.0-hdf63c60_3.tar.bz2           6.1 MB
s

Push the image to the docker repo.

In [12]:
!docker push $image_name

The push refers to repository [docker.io/mabouatmicrosoft/mlbaiht]

b9f8145e: Preparing 
8e663683: Preparing 
9b7f892f: Preparing 
9e19929c: Preparing 
b2f378bb: Preparing 
afdbe580: Preparing 
43c86cbc: Preparing 
b9f8145e: Pushed   1.203GB/1.159GBntu KPushing  410.6MB/1.159GBlatest: digest: sha256:28fe94a3d74a56e99be0f69f7021aaeeae8d6dccce193f310fd2d58cad826ea4 size: 1989


We can now test our image locally.